# RK Interpolation

This document includes Python codes that conduct Regression Kriging (RK) Interpolation on each waterbody, including Guana Tolomato Matanzas (GTM), Estero Bay (EB), Charlotte Harbor (CH), Biscayne Bay (BB), Big Bend Seagrasses (BBS).
parameters, including Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), Secchi (Secc_m), Total Nitrogen (TN_mgl) in arcpy environment.

The analysis is conducted in the separate managed parameters of Total Nitrogen (TN_mgl), Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), and Secchi (Secc_m) in arcpy environment.

* [1. Data Preprocess](#preprocessing)
* [2. Generate Shapefiles](#create_shp)
* [3. Regression Kriging for All Stations](#rk_all)

In [99]:
import pandas as pd
import numpy  as np
import arcpy
from arcpy.sa import *
import os, time, math, importlib, sys
path = r'../git/misc'
sys.path.insert(0, path)
import RK
# !install conda install conda-forge::pyproj
import pyproj,csv

importlib.reload(RK)

import warnings
warnings.filterwarnings('ignore')

# 1. Preprocessing <a class="anchor" id="preprocessing"></a>

## If you are the first time to run this code, you could run the following cell. 
## If you have already generated the combining csv, you could skip this cell and just run the next cell.

In [100]:
# gis_path = r'E:/Projects/SEACAR_WQ_2024/GIS_Data/'
gis_path = r"F:/SEACAR_WQ_2024/GIS_Data/"

dfDis = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-Jan-16.csv', low_memory=False)
dfCon = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Jan-16.csv', low_memory=False)

dfAll = pd.concat([dfDis, dfCon], ignore_index=True)

### Include the time period from 9 am to 17 pm in a day

In [101]:
# Convert string to datetime
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate'], format='%b %d %Y %I:%M%p')

# Include date from 9:00 am to 17:00 pm
start_time = '09:00'
end_time = '17:00'

dfAllTime = dfAll[dfAll['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]
dfAllTime.head()

,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,5062527,5014,Salinity,ppt,GTMMKNUT,Field,2017-08-03 11:24:00,2017,8,Surface,0.34,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Summer
1,5062528,5014,Salinity,ppt,GTMMKNUT,Field,2017-09-20 09:06:00,2017,9,Surface,0.32,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
2,5062529,5014,Secchi Depth,m,GTMMKNUT,Field,2017-11-02 13:11:00,2017,11,Surface,1.20,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
3,5062606,5014,Salinity,ppt,GTMMKNUT,Field,2017-10-18 12:52:00,2017,10,Surface,0.34,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
4,5062607,5014,Salinity,ppt,GTMMKNUT,Field,2018-04-24 10:56:00,2018,4,Surface,0.33,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Spring


In [102]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

# Set input parameters
waterbody_names = [
    'Guana Tolomato Matanzas',
    'Estero Bay',
    'Charlotte Harbor',
    'Biscayne Bay',
    'Big Bend Seagrasses'
]

covariates_dict = {
    "GTM":"LDI",
    "EB":"bathymetry+LDI+popden",
    "CH":"bathymetry+LDI+popden+water_flow_wet",
    "BB":"bathymetry+LDI+popden",
    "BBS":"bathymetry+LDI"
}

parameter_names = ['Dissolved Oxygen', 'Salinity', 'Secchi Depth', 'Total Nitrogen', 'Turbidity', 'Water Temperature']
# years = unique_years
seasons = ['Fall', 'Spring', 'Summer', 'Winter']
# shp_folder = gis_path + r"shapefiles_All"
shp_folder = gis_path + r"shapefiles"

# 2. Generate Shapefiles<a class="anchor" id="create_shp"></a>
## First, you need to aggregate the repeatly observation data into one value in the time period.

In [103]:
dfAll_Mean = dfAllTime.groupby(['WaterBody','ParameterName','ParameterUnits', 'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym'])["ResultValue"].agg("mean").reset_index()

# Save aggregated data to a csv file
dfAll = dfAll_Mean

### Convert coordinate system to EPSG: 3086

In [104]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
dfAll[['x', 'y']] = dfAll.apply(transform_coordinates, axis=1)

In [105]:
dfAll.to_csv(gis_path + 'OEAT_All_WQ-2024-Jan-16.csv', index=False)

## Generate the shapefiles of input points for each waterbody, parameter, year, and season

#### Fill NaN RowID with unique ID

In [106]:
RK.fill_nan_rowids(dfAll, 'RowID')

# Keep RowID as integer
dfAll['RowID'] = dfAll['RowID'].astype(int)
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y,RowID
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.256914,587084.624223,1
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.462864,645279.548973,2
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489729.880220,566493.097579,3
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.256914,587084.624223,4
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.462864,645279.548973,5
...,...,...,...,...,...,...,...,...,...,...,...,...
24379,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,29.906941,-81.298931,GTM,28.100000,660458.333777,658403.424703,24380
24380,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.586095,671395.945419,24381
24381,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653380.952596,671518.961956,24382
24382,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654940.976043,672348.548670,24383


In [121]:
# Read the season table
seasons_all = pd.read_csv(gis_path + 'Seasons_all.csv', low_memory=False)

# Merge interested with latitude and longitude columns
seasons_all_coord = RK.merge_with_lat_long(seasons_all, dfAll)
seasons_all_coord

,Unnamed: 0,WaterBody,Year,Season,Parameter,Filename,NumDataPoints,RMSE,ME,x,y,RowID,ResultValue
0,0,Guana Tolomato Matanzas,2015,Fall,Total Nitrogen,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
1,1,Guana Tolomato Matanzas,2015,Winter,Total Nitrogen,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
2,2,Guana Tolomato Matanzas,2016,Spring,Total Nitrogen,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
3,3,Guana Tolomato Matanzas,2016,Summer,Total Nitrogen,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
4,4,Guana Tolomato Matanzas,2016,Fall,Total Nitrogen,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,239,Big Bend Seagrasses,2022,Spring,Water Temperature,NaN,NaN,NaN,NaN,374832.099897,689623.362197,4393,20.533333
4683,239,Big Bend Seagrasses,2022,Spring,Water Temperature,NaN,NaN,NaN,NaN,371015.090649,692081.211171,4394,20.740000
4684,239,Big Bend Seagrasses,2022,Spring,Water Temperature,NaN,NaN,NaN,NaN,401894.595993,699335.031753,4395,19.500000
4685,239,Big Bend Seagrasses,2022,Spring,Water Temperature,NaN,NaN,NaN,NaN,401457.362103,702258.799507,4396,20.500000


### Create CSV files

In [49]:
# Clean the shapefile folder if necessary
# RK.delete_all_files(shp_folder)
# Print number of data points in each shapefile
RK.create_shp_season(seasons_all_coord, shp_folder)

Number of data rows for BBS, DO_mgl, 2020, Fall: 26
Shapefile for BBS: DO_mgl for year 2020 and season Fall has been saved as SHP_BBS_DO_mgl_2020_Fall.shp
Number of data rows for BBS, Sal_ppt, 2020, Fall: 21
Shapefile for BBS: Sal_ppt for year 2020 and season Fall has been saved as SHP_BBS_Sal_ppt_2020_Fall.shp
Number of data rows for BBS, Secc_m, 2020, Fall: 27
Shapefile for BBS: Secc_m for year 2020 and season Fall has been saved as SHP_BBS_Secc_m_2020_Fall.shp
Number of data rows for BBS, TN_mgl, 2020, Fall: 23
Shapefile for BBS: TN_mgl for year 2020 and season Fall has been saved as SHP_BBS_TN_mgl_2020_Fall.shp
Number of data rows for BBS, Turb_ntu, 2020, Fall: 26
Shapefile for BBS: Turb_ntu for year 2020 and season Fall has been saved as SHP_BBS_Turb_ntu_2020_Fall.shp
Number of data rows for BBS, T_c, 2020, Fall: 26
Shapefile for BBS: T_c for year 2020 and season Fall has been saved as SHP_BBS_T_c_2020_Fall.shp
Number of data rows for BBS, DO_mgl, 2020, Summer: 26
Shapefile for BB

Shapefile for BB: Turb_ntu for year 2021 and season Fall has been saved as SHP_BB_Turb_ntu_2021_Fall.shp
Number of data rows for BB, T_c, 2021, Fall: 83
Shapefile for BB: T_c for year 2021 and season Fall has been saved as SHP_BB_T_c_2021_Fall.shp
Number of data rows for BB, DO_mgl, 2021, Summer: 83
Shapefile for BB: DO_mgl for year 2021 and season Summer has been saved as SHP_BB_DO_mgl_2021_Summer.shp
Number of data rows for BB, Sal_ppt, 2021, Summer: 61
Shapefile for BB: Sal_ppt for year 2021 and season Summer has been saved as SHP_BB_Sal_ppt_2021_Summer.shp
Number of data rows for BB, Secc_m, 2021, Summer: 1
Shapefile for BB: Secc_m for year 2021 and season Summer has been saved as SHP_BB_Secc_m_2021_Summer.shp
Number of data rows for BB, TN_mgl, 2021, Summer: 76
Shapefile for BB: TN_mgl for year 2021 and season Summer has been saved as SHP_BB_TN_mgl_2021_Summer.shp
Number of data rows for BB, Turb_ntu, 2021, Summer: 61
Shapefile for BB: Turb_ntu for year 2021 and season Summer has 

Shapefile for CH: Sal_ppt for year 2016 and season Winter has been saved as SHP_CH_Sal_ppt_2016_Winter.shp
Number of data rows for CH, Secc_m, 2016, Winter: 8
Shapefile for CH: Secc_m for year 2016 and season Winter has been saved as SHP_CH_Secc_m_2016_Winter.shp
Number of data rows for CH, TN_mgl, 2016, Winter: 8
Shapefile for CH: TN_mgl for year 2016 and season Winter has been saved as SHP_CH_TN_mgl_2016_Winter.shp
Number of data rows for CH, Turb_ntu, 2016, Winter: 11
Shapefile for CH: Turb_ntu for year 2016 and season Winter has been saved as SHP_CH_Turb_ntu_2016_Winter.shp
Number of data rows for CH, T_c, 2016, Winter: 11
Shapefile for CH: T_c for year 2016 and season Winter has been saved as SHP_CH_T_c_2016_Winter.shp
Number of data rows for CH, DO_mgl, 2017, Fall: 3
Shapefile for CH: DO_mgl for year 2017 and season Fall has been saved as SHP_CH_DO_mgl_2017_Fall.shp
Number of data rows for CH, Sal_ppt, 2017, Fall: 3
Shapefile for CH: Sal_ppt for year 2017 and season Fall has been

Shapefile for EB: Turb_ntu for year 2017 and season Spring has been saved as SHP_EB_Turb_ntu_2017_Spring.shp
Number of data rows for EB, T_c, 2017, Spring: 3
Shapefile for EB: T_c for year 2017 and season Spring has been saved as SHP_EB_T_c_2017_Spring.shp
Number of data rows for EB, DO_mgl, 2017, Summer: 3
Shapefile for EB: DO_mgl for year 2017 and season Summer has been saved as SHP_EB_DO_mgl_2017_Summer.shp
Number of data rows for EB, Sal_ppt, 2017, Summer: 3
Shapefile for EB: Sal_ppt for year 2017 and season Summer has been saved as SHP_EB_Sal_ppt_2017_Summer.shp
No valid data found for area: EB, parameter: Secc_m, year: 2017, and season: Summer
No valid data found for area: EB, parameter: TN_mgl, year: 2017, and season: Summer
Number of data rows for EB, Turb_ntu, 2017, Summer: 3
Shapefile for EB: Turb_ntu for year 2017 and season Summer has been saved as SHP_EB_Turb_ntu_2017_Summer.shp
Number of data rows for EB, T_c, 2017, Summer: 3
Shapefile for EB: T_c for year 2017 and season

Shapefile for GTM: T_c for year 2017 and season Spring has been saved as SHP_GTM_T_c_2017_Spring.shp
Number of data rows for GTM, DO_mgl, 2017, Summer: 15
Shapefile for GTM: DO_mgl for year 2017 and season Summer has been saved as SHP_GTM_DO_mgl_2017_Summer.shp
Number of data rows for GTM, Sal_ppt, 2017, Summer: 16
Shapefile for GTM: Sal_ppt for year 2017 and season Summer has been saved as SHP_GTM_Sal_ppt_2017_Summer.shp
Number of data rows for GTM, Secc_m, 2017, Summer: 9
Shapefile for GTM: Secc_m for year 2017 and season Summer has been saved as SHP_GTM_Secc_m_2017_Summer.shp
Number of data rows for GTM, TN_mgl, 2017, Summer: 13
Shapefile for GTM: TN_mgl for year 2017 and season Summer has been saved as SHP_GTM_TN_mgl_2017_Summer.shp
Number of data rows for GTM, Turb_ntu, 2017, Summer: 4
Shapefile for GTM: Turb_ntu for year 2017 and season Summer has been saved as SHP_GTM_Turb_ntu_2017_Summer.shp
Number of data rows for GTM, T_c, 2017, Summer: 17
Shapefile for GTM: T_c for year 2017


# 3. Regression Kriging for both continuous and discrete data<a class="anchor" id="rk_all"></a>

## Loop for all parameters


### Clean the output folder

In [55]:
out_raster_floder = gis_path + "raster_output_rk/"
out_ga_folder     = gis_path + "ga_output_rk/"
diagnostic_folder = gis_path + "diagnostic_rk/"
std_error_folder  = gis_path + "std_error_rk/"
RK.delete_all_files(out_raster_floder)
RK.delete_all_files(out_ga_folder)
RK.delete_all_files(diagnostic_folder)
RK.delete_all_files(std_error_folder)

In [133]:
# Write the output in a csv file
with open(gis_path+"rk_results_temp.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header line
    cols = list(seasons_all.columns)
    cols.append('covariates')
    csv_writer.writerow(cols)
    
#     for i in seasons_all.index:
    for i in range(8,19):
        s_time =time.time() 
        process,rmse,me,count,file_loc = RK.rk_interpolation(method = "rk",
                                           radius = 10000,
                                           folder_path = gis_path,
                                           waterbody = area_shortnames[seasons_all.iloc[i]["WaterBody"]],
                                           parameter = param_shortnames[seasons_all.iloc[i]["Parameter"]],
                                           year      = seasons_all.iloc[i]["Year"],
                                           season    = seasons_all.iloc[i]['Season'],
                                           covariates= covariates_dict[area_shortnames[seasons_all.iloc[i]["WaterBody"]]],
                                           out_raster_folder = out_raster_floder,
                                           out_ga_folder     = out_ga_folder,
                                           std_error_folder  = std_error_folder,                  
                                           diagnostic_folder = diagnostic_folder)
        e_time =time.time()

        print(f"{int(e_time-s_time)} seconds elapsed for processing {count} points in {i}th row: RMSE: {rmse}, ME: {me}, file exported to {file_loc}")
        csv_writer.writerow([seasons_all.iloc[i]["WaterBody"], 
                             seasons_all.iloc[i]["Year"],
                             seasons_all.iloc[i]['Season'],
                             param_shortnames[seasons_all.iloc[i]["Parameter"]],
                             file_loc, count, rmse, me,
                             covariates_dict[area_shortnames[seasons_all.iloc[i]["WaterBody"]]]])
        if i%10 == 0: csvfile.flush() # flush the csv file in every 20 rows.
#         seasons_all['RMSE'][i:i+1] = rmse
#         seasons_all['ME'][i:i+1] = me
#         seasons_all['NumDataPoints'][i:i+1] = count
#         seasons_all['Filename'][i:i+1] = file_loc
#     seasons_all.to_csv(gis_path+"result_RK_all.csv")

No data for RK interpolation in SHP_EB_TN_mgl_2016_Summer.shp, skipping
0 seconds elapsed for processing 0 points in 8th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_TN_mgl_2016_Fall.shp, skipping
0 seconds elapsed for processing 0 points in 9th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_TN_mgl_2016_Winter.shp, skipping
0 seconds elapsed for processing 0 points in 10th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_TN_mgl_2017_Spring.shp, skipping
0 seconds elapsed for processing 0 points in 11th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_TN_mgl_2017_Summer.shp, skipping
0 seconds elapsed for processing 0 points in 12th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_TN_mgl_2017_Fall.shp, skipping
0 seconds elapsed for processing 0 points in 13th row: RMSE: nan, ME: nan, file exported to na

### Select the abnormal value from the generated results, here we defined that if the RMSE is too low (<-10000), it is the abnormal value.

In [124]:
error_row = pd.read_csv(gis_path+"rk_results_temp.csv")
error_row_new = error_row.loc[error_row["RMSE"]<-10000]
error_row_new = error_row_new.drop("Unnamed: 0",axis=1)

In [89]:
error_row_new.to_csv(gis_path+"temp_error.csv")

### Reset the smooth_radius from 10000 to 50000

In [134]:
with open(gis_path+"temp_error.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    cols = list(error_row_new.columns)
    cols.append('covariates')
    csv_writer.writerow(cols)
    
    for i in error_row_new.index:
#     for i in range(46,48):
        s_time =time.time() 
        process,rmse,me,count,file_loc = RK.rk_interpolation(method = "rk",
                                           radius = 50000,
                                           folder_path = gis_path,
                                           waterbody = area_shortnames[error_row_new.iloc[i]["WaterBody"]],
                                           parameter = param_shortnames[error_row_new.iloc[i]["Parameter"]],
                                           year      = error_row_new.iloc[i]["Year"],
                                           season    = error_row_new.iloc[i]['Season'],
                                           covariates= covariates_dict[area_shortnames[error_row_new.iloc[i]["WaterBody"]]],
                                           out_raster_folder = out_raster_floder,
                                           out_ga_folder     = out_ga_folder,
                                           std_error_folder  = std_error_folder,
                                           diagnostic_folder = diagnostic_folder)
        e_time =time.time()

        print(f"{int(e_time-s_time)} seconds elapsed for processing {count} points in {i}th row: RMSE: {rmse}, ME: {me}, file exported to {file_loc}")
        csv_writer.writerow([error_row_new.iloc[i]["WaterBody"], 
                             error_row_new.iloc[i]["Year"],
                             error_row_new.iloc[i]['Season'],
                             param_shortnames[error_row_new.iloc[i]["Parameter"]],
                             file_loc, count, rmse, me,
                             covariates_dict[area_shortnames[error_row_new.iloc[i]["WaterBody"]]]])
        if i%10 == 0: csvfile.flush()

### Merge the revised results into original results

In [135]:
original_file = pd.read_csv(gis_path+"rk_results_temp.csv")
error_row_file= pd.read_csv(gis_path+"temp_error.csv")

In [74]:
original_file_error_list = list(original_file[original_file["RMSE"]<-10000]["Unnamed: 0"])

In [76]:
abnormal_drop = original_file.drop(index=original_file_error_list) 

In [78]:
new_file = pd.concat([abnormal_drop,error_row_file])

In [ ]:
new_file = new_file.drop(["Unnamed: 0"],axis=1).reset_index(drop=True)

### Export the final results

In [86]:
new_file.to_csv(gis_path+"rk_results.csv")